# Similarity Judgement

In [1]:
#Via Gmail API
import base64
import httplib2
import email
from email.mime.text import MIMEText

from apiclient.discovery import build
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import run

SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'Gmail API Python Quickstart'


In [2]:
# Path to the client_secret.json file downloaded from the Developer Console
#CLIENT_SECRET_FILE = 'client_secret_william1121.json'
CLIENT_SECRET_FILE = 'client_secret.json'


# Check https://developers.google.com/gmail/api/auth/scopes for all available scopes
OAUTH_SCOPE = 'https://www.googleapis.com/auth/gmail.readonly'

# Location of the credentials storage file
STORAGE = Storage('gmail.storage')

# Start the OAuth flow to retrieve credentials
flow = flow_from_clientsecrets(CLIENT_SECRET_FILE, scope=OAUTH_SCOPE)
http = httplib2.Http()

# Try to retrieve credentials from storage or run the flow to generate them
credentials = STORAGE.get()
if credentials is None or credentials.invalid:
  credentials = run(flow, STORAGE, http=http)

# Authorize the httplib2.Http object with our credentials
http = credentials.authorize(http)

# Build the Gmail service from discovery
gmail_service = build('gmail', 'v1', http=http)


# Get all messages
results = gmail_service.users().messages().list(userId='me',q='').execute();
all_mes = [];
if 'messages' in results:
    all_mes.extend(results['messages'])

while 'nextPageToken' in results:
    page_token = results['nextPageToken']
    results = gmail_service.users().messages().list(userId='me',q='', pageToken=page_token).execute();
    all_mes.extend(results['messages'])

N = len(all_mes)
print N


190


In [3]:
#play with single message. Just for testing
mes_id = all_mes[10]['id'];
msg = gmail_service.users().messages().get(userId='me', id=mes_id, format='full').execute();
response = msg["payload"]


In [4]:
#retrieve all the responses
import re
result_list = [];
valid_response = 0;
for mes in all_mes:
    mes_id = mes['id'];
    msg = gmail_service.users().messages().get(userId='me', id=mes_id, format='full').execute();
    response = msg["payload"]
    #check headers
    headers = response["headers"];
    check_header = True;
    for item in headers:
        if item["name"] == "From":
            if item["value"]!='<ftvision1121@gmail.com>':
                check_header = False;
                break;
    if check_header == False:
        continue;
    #mime_msg = email.message_from_string(msg_str)
    k = 0;
    valid_response += 1;
    content = response["parts"][0]["body"]
    response_raw = content["data"];
    size = content["size"];
    response = base64.urlsafe_b64decode(response_raw.encode('ASCII')).split("\n")
    picture1 = re.search('[0-9]*-[0-9]*.jpg', response[0]).group(0);
    picture2 = re.search('[0-9]*-[0-9]*.jpg', response[1]).group(0);
    rating = int(response[2][-1])
    position = -1;
    for i in range(len(result_list)):
        if (result_list[i]["picture1"] == picture1) and (result_list[i]["picture2"]==picture2):
            position = i;
            break;
    if position >= 0:
        result_list[i]["rating"].append(rating)
    else:
        result_list.append({"picture1":picture1, "picture2":picture2, "rating":[rating]});
            
print "Totally %d responses:" % valid_response



Totally 186 responses:


In [5]:
for item in result_list:
    get_ans = [];
    for key in item:
        get_ans.append(item[key])
    print get_ans;
print len(result_list)


['5-2.jpg', [1, 2, 5, 3, 4, 6, 6, 1, 2, 1], '5-1.jpg']
['3-1.jpg', [1, 4, 4, 2, 4, 4], '3-2.jpg']
['6-1.jpg', [5, 6, 7, 5], '6-2.jpg']
['10-2.jpg', [5, 3, 5, 2, 5, 6, 4, 6], '10-1.jpg']
['8-1.jpg', [4, 3, 2, 1, 3, 3, 3, 2, 4, 5, 4, 3], '8-2.jpg']
['11-2.jpg', [5, 5, 5, 7, 1, 6], '11-1.jpg']
['8-2.jpg', [7, 1, 2, 2, 2, 2, 2, 1, 4, 3, 4], '8-1.jpg']
['9-1.jpg', [1, 5, 5], '9-2.jpg']
['12-1.jpg', [1, 1, 1, 3, 5, 5, 6, 4, 6], '12-2.jpg']
['11-1.jpg', [5, 1, 6, 4, 2, 4, 5, 5, 4], '11-2.jpg']
['4-1.jpg', [3, 3, 4, 4, 3, 5, 5, 4, 5, 2, 1], '4-2.jpg']
['1-2.jpg', [3, 5, 1, 3, 6], '1-1.jpg']
['2-2.jpg', [5, 5, 4, 3, 3, 3, 5, 7, 5, 5, 5, 5], '2-1.jpg']
['7-2.jpg', [3, 5, 3, 1, 4, 4, 2, 1, 1, 1, 5, 3, 1], '7-1.jpg']
['3-2.jpg', [4, 4, 7, 7, 4], '3-1.jpg']
['9-2.jpg', [7, 6, 6, 6, 6, 5, 7], '9-1.jpg']
['5-1.jpg', [1, 1, 7, 3, 3, 4, 4, 5], '5-2.jpg']
['2-1.jpg', [5, 3, 3, 6, 6, 2], '2-2.jpg']
['4-2.jpg', [4, 2, 5, 3, 2, 3, 3], '4-1.jpg']
['7-1.jpg', [3, 4, 2, 5, 2, 6], '7-2.jpg']
['12-2.jpg', [3, 6

In [6]:
#sort results and have a look at responses
sort_result = sorted(result_list, key=lambda k: k['picture1']) 
for item in sort_result:
    print "Picture 1: %-8s, Picture 2: %-8s, rating: %r" % (item["picture1"], item["picture2"], item["rating"])

Picture 1: 1-1.jpg , Picture 2: 1-2.jpg , rating: [3, 5, 1, 3, 6]
Picture 1: 1-2.jpg , Picture 2: 1-1.jpg , rating: [2, 4, 1, 5, 2]
Picture 1: 10-1.jpg, Picture 2: 10-2.jpg, rating: [5, 3, 5, 2, 5, 6, 4, 6]
Picture 1: 10-2.jpg, Picture 2: 10-1.jpg, rating: [7, 4, 4, 5, 5, 5, 6, 6]
Picture 1: 11-1.jpg, Picture 2: 11-2.jpg, rating: [5, 5, 5, 7, 1, 6]
Picture 1: 11-2.jpg, Picture 2: 11-1.jpg, rating: [5, 1, 6, 4, 2, 4, 5, 5, 4]
Picture 1: 12-1.jpg, Picture 2: 12-2.jpg, rating: [3, 6, 6, 3, 5, 2, 6, 1]
Picture 1: 12-2.jpg, Picture 2: 12-1.jpg, rating: [1, 1, 1, 3, 5, 5, 6, 4, 6]
Picture 1: 2-1.jpg , Picture 2: 2-2.jpg , rating: [5, 5, 4, 3, 3, 3, 5, 7, 5, 5, 5, 5]
Picture 1: 2-2.jpg , Picture 2: 2-1.jpg , rating: [5, 3, 3, 6, 6, 2]
Picture 1: 3-1.jpg , Picture 2: 3-2.jpg , rating: [4, 4, 7, 7, 4]
Picture 1: 3-2.jpg , Picture 2: 3-1.jpg , rating: [1, 4, 4, 2, 4, 4]
Picture 1: 4-1.jpg , Picture 2: 4-2.jpg , rating: [4, 2, 5, 3, 2, 3, 3]
Picture 1: 4-2.jpg , Picture 2: 4-1.jpg , rating: [3, 3